In [1]:
import pandas as pd

In [2]:
# MODIFICATION : Ajoute le dictionnaire de types ici
types_fixes = {
    "Code postal de l'établissement": str,
    "Code du département de l'établissement": str,
    "SIREN": str,
    "Code commune de l'établissement": str
}

df = pd.read_csv("data.csv", dtype=types_fixes, sep=";", low_memory=False)
pd.set_option('display.max_columns', None)
df.head()

,SIREN,NIC,SIRET,Statut de diffusion de l'établissement,Date de création de l'établissement,Tranche de l'effectif de l'établissement,Tranche de l'effectif de l'établissement triable,Année de la tranche d'effectif de l'établissement,Activité principale de l'établissement,Date de la dernière mise à jour de l'établissement,Etablissement siège,Nombre de periodes de l'établissement,Complément d'adresse de l'établissement,Numéro de voie de l'établissement,Indice de répétition de l'établissement,Type de voie de l'établissement,Libellé de la voie de l'établissement,Code postal de l'établissement,Commune de l'établissement,Libellé de la commune de l'établissement à l'étranger,Distribution spéciale de l'établissement,Code commune de l'établissement,Code cedex de l'établissement,Libellé cedex de l'établissement,Code du pays de l'établissement étranger,Libellé du pays de l'établissement étranger,Complément d'adresse de l'établissement 2,Numero de voie de l'établissement 2,Indice de répétition de l'établissement 2,Type de voie de l'établissement 2,Libellé de la voie de l'établissement 2,Code postal de l'établissement 2,Libellé de la commune de l'établissement 2,Libellé de la commune de l'établissement étranger 2,Distribution spéciale de l'établissement 2,Code de la commune de l'établissement 2,Code cedex de l'établissement 2,Libellé cedex de l'établissement 2,Code pays de l'établissement,Libellé du pays de l'établissement étranger.1,Date du début de la période de l'établissement,Etat administratif de l'établissement,Enseigne de l'établissement 1,Enseigne de l'établissement 2,Enseigne de l'établissement 3,Dénomination usuelle de l'établissement,Activité principale de l'établissement.1,Nomenclature principale de l'établissement,Caractère employeur de l'établissement,Statut de diffusion de l'unité légale,Unité légale purgée,Date de création de l'unité légale,Sigle de l'unité légale,Civilité de la personne physique,Prénom de la personne physique 1,Prénom de la personne physique 2,Prénom de la personne physique 3,Prénom de la personne physique 4,Prénom usuel de la personne physique,Pseudonyme de la personne physique,Identifiant association de l'unité légale,Tranche de l'effectif de l'unité légale,Tranche de l'effectif de l'unité légale triable,Année de la tranche de l'effectif de l'unité légale,Date du dernier traitement de l'unité légale,Nombre de périodes de l'unité légale,Catégorie de l'entreprise,Année de la catégorie de l'entreprise,Date de début de l'unité légale,Etat administratif de l'unité légale,Nom de la personne physique,Nom d'usage de la personne physique,Dénomination de l'unité légale,Dénomination usuelle de l'unité légale 1,Dénomination usuelle de l'unité légale 2,Dénomination usuelle de l'unité légale 3,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Nomenclature de l'activité principale de l'unité légale,NIC du siège de l'unité légale,Economie sociale et solidaire unité légale,Société à mission unité légale,Caractère employeur de l'unité légale,Code EPCI de l'établissement,Libellé de l'EPCI de l'établissement,Code de l'arrondissement de l'établissement,Code du département de l'établissement,Département de l'établissement,Code de la région de l'établissement,Région de l'établissement,Section de l'établissement,Sous-section de l'établissement,Division de l'établissement,Groupe de l'établissement,Classe de l'établissement,Section de l'unité légale,Sous-section de l'unité légale,Division de l'unité légale,Groupe de l'unité légale,Classe de l'unité légale,Nature juridique de l'unité légale,Première ligne de l'adressage,Adresse de l'établissement,SIRET du siège de l'unité légale,Date de fermeture de l'établissement,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,912239670,17,91223967000017,O,2022-04-05,3 à 5 salariés,2,2023.0,4391AZ,2025-02-05T03:21:54+01:00,oui,2,NaN,220.0,A,Rue,DE FONTGRAVE,26740,MONTBOUCHER-SUR-JABRON,NaN,NaN,26191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
df.shape

(2295497, 107)

In [4]:
df.dtypes

SIREN                                      object
NIC                                         int64
SIRET                                       int64
Statut de diffusion de l'établissement     object
Date de création de l'établissement        object
                                           ...   
Adresse de l'établissement                 object
SIRET du siège de l'unité légale          float64
Date de fermeture de l'établissement      float64
Date de fermeture de l'unité légale        object
Géolocalisation de l'établissement         object
Length: 107, dtype: object

---

#### Vérification des SIREN, NIC pour valider les drop

In [ ]:
# On s'assure que les colonnes sont comparables (en string sans le .0)
siret_str = df["SIRET"].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(14)
siege_str = df["SIRET du siège de l'unité légale"].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(14)

# 1. Vérification de la concordance SIRET ↔ SIRET du siège

siret_ok = (siret_str == siege_str)

# 2. Vérification de la cohérence SIREN ↔ SIRET
siren_nunique = df["SIREN"].nunique()
siret_nunique = df["SIRET"].nunique()

# Résumé
print(f"Nombre de SIREN uniques : {siren_nunique}")
print(f"Nombre de SIRET uniques : {siret_nunique}")
print("Cohérence SIREN / SIRET (SIREN <= SIRET) :", siren_nunique <= siret_nunique)

# Optionnel : combien d'établissements sont des sièges ?
print(f"Nombre de sièges identifiés : {siret_ok.sum()}")

Nombre de SIREN uniques : 2074980
Nombre de SIRET uniques : 2295497
Cohérence SIREN / SIRET (SIREN <= SIRET) : True
Nombre de sièges identifiés : 2064481


In [ ]:
# On crée un masque booléen sécurisé

is_siege = (
    df["SIRET"].astype(str).str.replace(r'\.0$', '', regex=True) == 
    df["SIRET du siège de l'unité légale"].astype(str).str.replace(r'\.0$', '', regex=True)
)

sieges_par_siren = (
    df[is_siege]
    .groupby("SIREN")
    .size()
)

print("Tous les SIREN ont un seul siège :", (sieges_par_siren == 1).all())


Tous les SIREN ont un seul siège : True


---

#### Drop des colonnes inutiles selon l'études des sociétés fermées

In [10]:
# Liste des colonnes à supprimer
colonnes_a_drop = [
    
    "Libellé cedex de l'établissement",
    "Numero de voie de l'établissement 2",
    "Distribution spéciale de l'établissement",
    "Libellé du pays de l'établissement étranger.1",
    "Prénom usuel de la personne physique",
    "Prénom de la personne physique 3",
    "Indice de répétition de l'établissement 2",
    "Complément d'adresse de l'établissement 2",
    "Code cedex de l'établissement",
    "Code pays de l'établissement",
    "Libellé cedex de l'établissement 2",
    "Code cedex de l'établissement 2",
    "Code de la commune de l'établissement 2",
    "Distribution spéciale de l'établissement 2",
    "Libellé de la commune de l'établissement étranger 2",
    "Libellé de la commune de l'établissement 2",
    "Code postal de l'établissement 2",
    "Libellé de la voie de l'établissement 2",
    "Type de voie de l'établissement 2",
    "Pseudonyme de la personne physique",
    "Prénom de la personne physique 1",
    "Civilité de la personne physique",
    "Prénom de la personne physique 4",
    "Caractère employeur de l'unité légale",
    "Première ligne de l'adressage",
    "Nom de la personne physique",
    "Nom d'usage de la personne physique",
    "Prénom de la personne physique 2",
    "Unité légale purgée",
    "Libellé de la commune de l'établissement à l'étranger",
    "Code du pays de l'établissement étranger",
    "Identifiant association de l'unité légale",
    "Libellé du pays de l'établissement étranger",
    "Dénomination usuelle de l'unité légale 3",
    "Enseigne de l'établissement 3",
    "Dénomination usuelle de l'unité légale 2",
    "Enseigne de l'établissement 2",
    "Société à mission unité légale",
    "Indice de répétition de l'établissement",
    "Dénomination usuelle de l'unité légale 1",
    "Sigle de l'unité légale",
    "Enseigne de l'établissement 1",
    "Complément d'adresse de l'établissement",
    "Année de la tranche d'effectif de l'établissement",
    "Dénomination usuelle de l'établissement",
    "Activité principale de l'établissement",
    "Année de la tranche de l'effectif de l'unité légale",
    "Etablissement siège",
    "Statut de diffusion de l'établissement",
    "Statut de diffusion de l'unité légale",
    "Tranche de l'effectif de l'établissement",
    "Tranche de l'effectif de l'établissement triable",
    "Numéro de voie de l'établissement",
    "Type de voie de l'établissement",
    "Libellé de la voie de l'établissement",
    "NIC",
    "SIRET",
    "NIC du siège de l'unité légale",
    "Activité principale de l'établissement.1",

    "Commune de l'établissement",
    "Caractère employeur de l'établissement",
    "Date du dernier traitement de l'unité légale",
    "Nombre de périodes de l'unité légale",
    "Etat administratif de l'établissement",

    "Code EPCI de l'établissement",
    "Libellé de l'EPCI de l'établissement",
    "Département de l'établissement",
    "Région de l'établissement",
    "Section de l'établissement",
    "Sous-section de l'établissement",
    "Division de l'établissement",
    "Groupe de l'établissement",
    "Adresse de l'établissement",
    "SIRET du siège de l'unité légale",
    "Date de fermeture de l'établissement",   


    "Tranche de l'effectif de l'unité légale triable", 
    "Catégorie de l'entreprise", 
    "Année de la catégorie de l'entreprise", 
    "Classe de l'établissement",
    "Section de l'unité légale",
    "Sous-section de l'unité légale",
    "Division de l'unité légale",
    "Groupe de l'unité légale",
    "Classe de l'unité légale",
    "Nombre de periodes de l'établissement",
    "Nomenclature de l'activité principale de l'unité légale",
    "Nomenclature principale de l'établissement",
    "Code de l'arrondissement de l'établissement",
    
]

# Supprimer les colonnes
df = df.drop(columns=colonnes_a_drop)

# Vérification de la nouvelle taille du dataset
print(f"Dimensions après nettoyage : {df.shape}")

Dimensions après nettoyage : (2295497, 19)


In [11]:
df.head()

,SIREN,Date de création de l'établissement,Date de la dernière mise à jour de l'établissement,Code postal de l'établissement,Code commune de l'établissement,Date du début de la période de l'établissement,Date de création de l'unité légale,Tranche de l'effectif de l'unité légale,Date de début de l'unité légale,Etat administratif de l'unité légale,Dénomination de l'unité légale,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Economie sociale et solidaire unité légale,Code du département de l'établissement,Code de la région de l'établissement,Nature juridique de l'unité légale,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,912239670,2022-04-05,2025-02-05T03:21:54+01:00,26740,26191,2022-12-12,2022-04-05,3 à 5 salariés,2022-12-12,Active,LES TOITURES PERENNES,5499,43.91A,N,26,84.0,Société à responsabilité limitée (sans autre i...,NaN,"44.547994, 4.796421"
1,500250303,2007-09-14,2025-02-04T03:24:17+01:00,94800,94076,2023-06-05,2007-09-14,Etablissement non employeur,2023-06-05,Active,NaN,1000,47.89Z,NaN,94,11.0,Entrepreneur individuel,NaN,"48.796614, 2.367839"
2,389091893,2019-01-01,2024-03-30T07:51:21+01:00,91270,91657,2019-01-01,1992-11-02,10 à 19 salariés,2019-01-01,Active,OCB,5499,43.99C,N,91,11.0,Société à responsabilité limitée (sans autre i...,NaN,"48.715618, 2.434367"
3,212505887,1996-12-01,2024-03-30T19:49:51+01:00,25380,25588,2008-01-01,1978-01-01,3 à 5 salariés,2017-12-31,Active,COMMUNE DE VAUCLUSE,7210,84.11Z,N,25,27.0,Commune et commune nouvelle,NaN,"47.258419, 6.685238"
4,200053825,2016-01-01,2024-03-22T16:40:57+01:00,50160,50601,2016-01-01,2016-01-01,50 à 99 salariés,2017-12-31,Active,COMMUNE DE TORIGNY-LES-VILLES,7210,84.11Z,N,50,28.0,Commune et commune nouvelle,NaN,"49.035001, -0.97898"


In [9]:
df.columns.to_list()

['SIREN',
 "Date de création de l'établissement",
 "Date de la dernière mise à jour de l'établissement",
 "Code postal de l'établissement",
 "Code commune de l'établissement",
 "Date du début de la période de l'établissement",
 "Date de création de l'unité légale",
 "Tranche de l'effectif de l'unité légale",
 "Date de début de l'unité légale",
 "Etat administratif de l'unité légale",
 "Dénomination de l'unité légale",
 "Catégorie juridique de l'unité légale",
 "Activité principale de l'unité légale",
 'Economie sociale et solidaire unité légale',
 "Code du département de l'établissement",
 "Code de la région de l'établissement",
 "Nature juridique de l'unité légale",
 "Date de fermeture de l'unité légale",
 "Géolocalisation de l'établissement"]

---

#### Filtrage des SAS et SARL

In [15]:
# 🔹 Étape 1 : filtrer uniquement SAS et SARL exacte
sas_sarl_values = [
    "SAS, société par actions simplifiée",
    "Société à responsabilité limitée (sans autre indication)"
]

df_filtered = df[df["Nature juridique de l'unité légale"].isin(sas_sarl_values)]
# 🔹 Étape 2 : vérification rapide

print("Formes juridiques retenues :")
print(df_filtered["Nature juridique de l'unité légale"].value_counts())

Formes juridiques retenues :
Nature juridique de l'unité légale
Société à responsabilité limitée (sans autre indication)    415885
SAS, société par actions simplifiée                         379243
Name: count, dtype: int64


In [16]:
df_filtered.shape

(795128, 19)

---

#### Examen des valeurs pour décisions à prendre

In [17]:
df_filtered.head()

,SIREN,Date de création de l'établissement,Date de la dernière mise à jour de l'établissement,Code postal de l'établissement,Code commune de l'établissement,Date du début de la période de l'établissement,Date de création de l'unité légale,Tranche de l'effectif de l'unité légale,Date de début de l'unité légale,Etat administratif de l'unité légale,Dénomination de l'unité légale,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Economie sociale et solidaire unité légale,Code du département de l'établissement,Code de la région de l'établissement,Nature juridique de l'unité légale,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,912239670,2022-04-05,2025-02-05T03:21:54+01:00,26740,26191,2022-12-12,2022-04-05,3 à 5 salariés,2022-12-12,Active,LES TOITURES PERENNES,5499,43.91A,N,26,84.0,Société à responsabilité limitée (sans autre i...,NaN,"44.547994, 4.796421"
2,389091893,2019-01-01,2024-03-30T07:51:21+01:00,91270,91657,2019-01-01,1992-11-02,10 à 19 salariés,2019-01-01,Active,OCB,5499,43.99C,N,91,11.0,Société à responsabilité limitée (sans autre i...,NaN,"48.715618, 2.434367"
7,888366747,2023-03-06,2025-07-20T04:05:42+02:00,30310,30344,2025-06-30,2020-08-11,1 ou 2 salariés,2023-03-06,Active,COMME UN DIADEME,5710,47.91A,N,30,76.0,"SAS, société par actions simplifiée",NaN,"43.743188, 4.206208"
10,953937455,2023-07-27,2024-03-30T15:49:14+01:00,75008,75108,2023-07-27,2023-07-27,Etablissement non employeur,2023-07-27,Active,WE CLEAN HOLDING PARNASSA,5710,66.30Z,N,75,11.0,"SAS, société par actions simplifiée",NaN,"48.873057, 2.316605"
11,840236111,2018-06-05,2024-03-30T04:07:14+01:00,94440,94070,2018-08-01,2018-06-05,1 ou 2 salariés,2018-08-01,Active,LEVIO,5710,96.02A,NaN,94,11.0,"SAS, société par actions simplifiée",NaN,"48.728308, 2.574656"


In [18]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

Date de fermeture de l'unité légale           99.999874
Economie sociale et solidaire unité légale    30.004477
Géolocalisation de l'établissement             0.889165
Code du département de l'établissement         0.093570
Code de la région de l'établissement           0.093570
Code postal de l'établissement                 0.073699
Code commune de l'établissement                0.010690
Dénomination de l'unité légale                 0.000252
dtype: float64


In [19]:
df_filtered['Economie sociale et solidaire unité légale'].value_counts()

Economie sociale et solidaire unité légale
N    552684
O      3870
Name: count, dtype: int64

##### Remplacement des Nan dans l'ESS par des N

In [20]:
df_filtered['Economie sociale et solidaire unité légale'] = (
    df_filtered['Economie sociale et solidaire unité légale']
    .fillna("N")
)


C:\Users\joelt\AppData\Local\Temp\ipykernel_16804\1554428496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Economie sociale et solidaire unité légale'] = (


In [21]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

Date de fermeture de l'unité légale       99.999874
Géolocalisation de l'établissement         0.889165
Code du département de l'établissement     0.093570
Code de la région de l'établissement       0.093570
Code postal de l'établissement             0.073699
Code commune de l'établissement            0.010690
Dénomination de l'unité légale             0.000252
dtype: float64


In [22]:
df_filtered['Date de fermeture de l\'unité légale'].value_counts()

Date de fermeture de l'unité légale
2025-05-20    1
Name: count, dtype: int64

In [23]:
df_filtered = df_filtered[df_filtered["Date de fermeture de l'unité légale"].isna()]


In [24]:
# Incrémation d'un Nan pour la Date de fermeture : les sociétés sont ouvertes
df_filtered["Date de fermeture de l'unité légale"].value_counts(dropna=False)


Date de fermeture de l'unité légale
NaN    795127
Name: count, dtype: int64

---

#### Remplacement de la date de fermeture par le 31 décembre 2025 en prévision du calcul de l'âge des sociétés.

In [25]:
df_filtered["Date de fermeture de l'unité légale"] = "2025-12-31"


In [26]:
df_filtered.head()

,SIREN,Date de création de l'établissement,Date de la dernière mise à jour de l'établissement,Code postal de l'établissement,Code commune de l'établissement,Date du début de la période de l'établissement,Date de création de l'unité légale,Tranche de l'effectif de l'unité légale,Date de début de l'unité légale,Etat administratif de l'unité légale,Dénomination de l'unité légale,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Economie sociale et solidaire unité légale,Code du département de l'établissement,Code de la région de l'établissement,Nature juridique de l'unité légale,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,912239670,2022-04-05,2025-02-05T03:21:54+01:00,26740,26191,2022-12-12,2022-04-05,3 à 5 salariés,2022-12-12,Active,LES TOITURES PERENNES,5499,43.91A,N,26,84.0,Société à responsabilité limitée (sans autre i...,2025-12-31,"44.547994, 4.796421"
2,389091893,2019-01-01,2024-03-30T07:51:21+01:00,91270,91657,2019-01-01,1992-11-02,10 à 19 salariés,2019-01-01,Active,OCB,5499,43.99C,N,91,11.0,Société à responsabilité limitée (sans autre i...,2025-12-31,"48.715618, 2.434367"
7,888366747,2023-03-06,2025-07-20T04:05:42+02:00,30310,30344,2025-06-30,2020-08-11,1 ou 2 salariés,2023-03-06,Active,COMME UN DIADEME,5710,47.91A,N,30,76.0,"SAS, société par actions simplifiée",2025-12-31,"43.743188, 4.206208"
10,953937455,2023-07-27,2024-03-30T15:49:14+01:00,75008,75108,2023-07-27,2023-07-27,Etablissement non employeur,2023-07-27,Active,WE CLEAN HOLDING PARNASSA,5710,66.30Z,N,75,11.0,"SAS, société par actions simplifiée",2025-12-31,"48.873057, 2.316605"
11,840236111,2018-06-05,2024-03-30T04:07:14+01:00,94440,94070,2018-08-01,2018-06-05,1 ou 2 salariés,2018-08-01,Active,LEVIO,5710,96.02A,N,94,11.0,"SAS, société par actions simplifiée",2025-12-31,"48.728308, 2.574656"


#### Nan restants à traiter dans le second notebook

In [27]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

Géolocalisation de l'établissement        0.889166
Code du département de l'établissement    0.093570
Code de la région de l'établissement      0.093570
Code postal de l'établissement            0.073699
Code commune de l'établissement           0.010690
Dénomination de l'unité légale            0.000252
dtype: float64


In [35]:
df_filtered.shape

(795127, 19)

---

In [29]:
df_filtered.dtypes

SIREN                                                  object
Date de création de l'établissement                    object
Date de la dernière mise à jour de l'établissement     object
Code postal de l'établissement                         object
Code commune de l'établissement                        object
Date du début de la période de l'établissement         object
Date de création de l'unité légale                     object
Tranche de l'effectif de l'unité légale                object
Date de début de l'unité légale                        object
Etat administratif de l'unité légale                   object
Dénomination de l'unité légale                         object
Catégorie juridique de l'unité légale                   int64
Activité principale de l'unité légale                  object
Economie sociale et solidaire unité légale             object
Code du département de l'établissement                 object
Code de la région de l'établissement                  float64
Nature j

In [ ]:
# 1. On règle le cas du Code Région (84.0 -> "84")
df_filtered["Code de la région de l'établissement"] = (
    df_filtered["Code de la région de l'établissement"]
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
)

# 2. Conversion en datetime (pour que le format Parquet les reconnaisse)

cols_dates = [
    "Date de création de l'établissement", 
    "Date de création de l'unité légale", 
    "Date de fermeture de l'unité légale"
]

for col in cols_dates:
    df_filtered[col] = pd.to_datetime(df_filtered[col], errors='coerce')

df_filtered.dtypes

SIREN                                                         object
Date de création de l'établissement                   datetime64[ns]
Date de la dernière mise à jour de l'établissement            object
Code postal de l'établissement                                object
Code commune de l'établissement                               object
Date du début de la période de l'établissement                object
Date de création de l'unité légale                    datetime64[ns]
Tranche de l'effectif de l'unité légale                       object
Date de début de l'unité légale                               object
Etat administratif de l'unité légale                          object
Dénomination de l'unité légale                                object
Catégorie juridique de l'unité légale                          int64
Activité principale de l'unité légale                         object
Economie sociale et solidaire unité légale                    object
Code du département de l'établisse

In [33]:
df_filtered.head()

,SIREN,Date de création de l'établissement,Date de la dernière mise à jour de l'établissement,Code postal de l'établissement,Code commune de l'établissement,Date du début de la période de l'établissement,Date de création de l'unité légale,Tranche de l'effectif de l'unité légale,Date de début de l'unité légale,Etat administratif de l'unité légale,Dénomination de l'unité légale,Catégorie juridique de l'unité légale,Activité principale de l'unité légale,Economie sociale et solidaire unité légale,Code du département de l'établissement,Code de la région de l'établissement,Nature juridique de l'unité légale,Date de fermeture de l'unité légale,Géolocalisation de l'établissement
0,912239670,2022-04-05,2025-02-05T03:21:54+01:00,26740,26191,2022-12-12,2022-04-05,3 à 5 salariés,2022-12-12,Active,LES TOITURES PERENNES,5499,43.91A,N,26,84,Société à responsabilité limitée (sans autre i...,2025-12-31,"44.547994, 4.796421"
2,389091893,2019-01-01,2024-03-30T07:51:21+01:00,91270,91657,2019-01-01,1992-11-02,10 à 19 salariés,2019-01-01,Active,OCB,5499,43.99C,N,91,11,Société à responsabilité limitée (sans autre i...,2025-12-31,"48.715618, 2.434367"
7,888366747,2023-03-06,2025-07-20T04:05:42+02:00,30310,30344,2025-06-30,2020-08-11,1 ou 2 salariés,2023-03-06,Active,COMME UN DIADEME,5710,47.91A,N,30,76,"SAS, société par actions simplifiée",2025-12-31,"43.743188, 4.206208"
10,953937455,2023-07-27,2024-03-30T15:49:14+01:00,75008,75108,2023-07-27,2023-07-27,Etablissement non employeur,2023-07-27,Active,WE CLEAN HOLDING PARNASSA,5710,66.30Z,N,75,11,"SAS, société par actions simplifiée",2025-12-31,"48.873057, 2.316605"
11,840236111,2018-06-05,2024-03-30T04:07:14+01:00,94440,94070,2018-08-01,2018-06-05,1 ou 2 salariés,2018-08-01,Active,LEVIO,5710,96.02A,N,94,11,"SAS, société par actions simplifiée",2025-12-31,"48.728308, 2.574656"


##### Export Parquet

In [ ]:
# # 3. Exportation finale en Parquet
# df_filtered.to_parquet("sas_sarl_ouvertes.parquet", index=False)